Модуль 2. Спам не пройдет

# Модуль 2. Спам не пройдет

In [1]:
from collections import Counter  #библиотека для словаря, который позволяет нам считать количество неизменяемых слов
import re  #библиотека для очистки слов от знаков препинания
import pandas as pd

In [2]:
# в заданной обучающей выборке писем SPAM and NOT_SPAM
# чтобы не менять их на строки вручную и чтобы Python не ругался при вводе, 
# зададим переменные SPAM and NOT_SPAM
SPAM = 1
NOT_SPAM = 0

In [3]:
#обучаюшая выборка со SPAM and NOT_SPAM письмами из задания 
train_data = [['Купите новое чистящее средство', SPAM],
              ['Купи мою новую книгу', SPAM],  
              ['Подари себе новый телефон', SPAM],
              ['Добро пожаловать и купите новый телевизор', SPAM],
              ['Привет давно не виделись', NOT_SPAM], 
              ['Довезем до аэропорта из пригорода всего за 399 рублей', SPAM], 
              ['Добро пожаловать в Мой Круг', NOT_SPAM],  
              ['Я все еще жду документы', NOT_SPAM],  
              ['Приглашаем на конференцию Data Science', NOT_SPAM],
              ['Потерял твой телефон напомни', NOT_SPAM],
              ['Порадуй своего питомца новым костюмом', SPAM]]  

In [4]:
# инициируем словари для слов из писем со SPAM and NOT_SPAM

dict_of_Spam_words = Counter()
dict_of_NOspam_words = Counter()

In [5]:
# функцию которая заполняет словари. Она будет принимать на вход письмо и его метку 
# — спам это или не спам и раскладывать по словарям считая вхождения

def calculate_word_frequencies(d_text, d_label):
    list_of_words = d_text.lower().strip().split(" ")  #делаем маленьким регистром и убираем пробелы
    if d_label == 1:
        for word in list_of_words:
            dict_of_Spam_words[word] += 1  #если письмо спам (SPAM==1) тогда заполняем словарь SPAM
    
    else:
        for word in list_of_words:
            dict_of_NOspam_words[word] += 1  #если письмо не спам (SPAM!=1) тогда заполняем словарь NoSPAM
    
    return

In [6]:
# инициируем переменные вероятностей A и not A — вероятность встретить спам и не встретить спам
# в задании предлагаю назвать эти переменные pA и pNotA, но я назвал более понятно для себя
freq_of_spam = 0.0
freq_of_NOspam = 0.0


In [7]:
# заполняем словари

count_of_letter_of_SPAM = 0  # инициализируем переменнные счетчиков для подчета писем SPAM and NOspam
count_of_letter_of_NOspam = 0

for elem in train_data:
    if elem[1] == 1:
        count_of_letter_of_SPAM += 1
    else:
        count_of_letter_of_NOspam += 1
    calculate_word_frequencies(*elem)

freq_of_spam = count_of_letter_of_SPAM / (count_of_letter_of_SPAM + count_of_letter_of_NOspam)
freq_of_NOspam = count_of_letter_of_NOspam / (count_of_letter_of_SPAM + count_of_letter_of_NOspam)

In [8]:
freq_of_spam

0.5454545454545454

In [9]:
freq_of_NOspam

0.45454545454545453

In [10]:
# Пишем функцию для обучения

def train(d_train_data):
    
    d_count_of_letter_of_SPAM = 0  # инициализируем переменнные счетчиков для подчета писем SPAM and NOspam
    d_count_of_letter_of_NOspam = 0
    
    for elem in d_train_data:
        global freq_of_spam, freq_of_NOspam  # пропишем глобальные переменные freq_of_spam, freq_of_NOspam
        if elem[1] == 1:
            d_count_of_letter_of_SPAM += 1
           
        else:
            d_count_of_letter_of_NOspam += 1
        calculate_word_frequencies(*elem)

    freq_of_spam = d_count_of_letter_of_SPAM / (d_count_of_letter_of_SPAM + d_count_of_letter_of_NOspam)
    freq_of_spam = round(freq_of_spam,4)
    # подсчитываем частоту SPAM писем на тренировочных данных
    # кол-во писем со спамом делим на общее кол-во писем
    # это в задании называется pA - вероятность спама вообще
    # округляем до 2 знака
    
    freq_of_NOspam = 1 - freq_of_spam
    # подсчитываем частоту NOT_SPAM писем как раницу 1-pA
    # это в задании pNotA - вероятность НЕ спама вообще
    # 100% вычитаем pA - так как pA + pNotA = 1 = 100%
    
    return print("system message: def train completed successfully")

In [11]:
freq_of_spam = 0.0
freq_of_NOspam = 0.0
dict_of_Spam_words = Counter()
dict_of_NOspam_words = Counter()
train(train_data)

system message: def train completed successfully


In [12]:
# инициализируем глабальные константы для расчета вероятностей 
alfa = 1  #параметр для сглаживания
N_spam = sum(dict_of_Spam_words.values())  # количество ВСЕХ слов входящих в тренировочную выборку писем SPAM
N_nospam = sum(dict_of_NOspam_words.values())  # количество ВСЕХ слов входящих в тренировочную выборку писем NOT_SPAM
NU_spam = len(dict_of_Spam_words)  # количество УНИКАЛЬНЫХ слов входящих в тренировочную выборку писем SPAM
NU_nospam = len(dict_of_NOspam_words)  # количество УНИКАЛЬНЫХ слов входящих в тренировочную выборку писем NOT_SPAM

In [13]:
# функцию подсчета вероятности вхождения слова Xi в класс d_label (SPAM or NOT_spam)
# или другими словами для каждого слова вероятность того, что это спам, и того, что это не спам
def calculate_P_Bi_A(d_word, d_label):
    if d_label == 1:
        temp_freq = (alfa + dict_of_Spam_words[d_word])/(alfa*NU_spam + N_spam)
        # print(f"{d_word} in {d_label} := {temp_freq}")
        return temp_freq
    else:
        temp_freq = (alfa + dict_of_NOspam_words[d_word])/(alfa* NU_nospam + N_nospam)
        # print(f"{d_word} in {d_label} := {temp_freq}")      
        return temp_freq

In [14]:
# проверка Антона 
sum([calculate_P_Bi_A(w,SPAM) for w in dict_of_Spam_words.keys()])

0.9999999999999994

In [15]:
# проверка Антона 
sum([calculate_P_Bi_A(w,NOT_SPAM) for w in dict_of_NOspam_words.keys()])

0.9999999999999996

In [16]:
#инициирование новых глобальных переменных исходя из тесттекста
    
def initiation_of_new_global_var(d_text):
    global N_spam, N_nospam, NU_spam, NU_nospam
    
    ws = set(re.sub(r"[!?]+$", "", elem) for elem in d_text.lower().strip().split(" "))
    NU_nospam = len(ws)
    NU_spam = NU_nospam
    N_spam = len(set(dict_of_Spam_words.keys()) & ws)
    N_nospam = len(set(dict_of_NOspam_words.keys()) & ws)
    return

In [17]:
# функция подсчитывает вероятность спама и неспама для строки
def calculate_P_B_A(d_text, d_label):
    
    initiation_of_new_global_var(d_text)
    
    result = 1
    for elem in d_text.lower().strip().split(" "):
        word = re.sub(r"[!?]+$", '', elem)
        result *= calculate_P_Bi_A(word, d_label)
    return result

In [18]:
def classify(email):
    global alfa,N_spam,N_nospam,M,freq_of_spam,freq_of_NOspam
    temp_freq = calculate_P_B_A(email, 1)
#     print(f"P_B_A in 1 := {temp_freq}")
    freq_of_hypothesis_Spam = freq_of_spam*temp_freq
    
    temp_freq = calculate_P_B_A(email, 0)
#     print(f"P_B_A in 0 := {temp_freq}")
    freq_of_hypothesis_NOspam = freq_of_NOspam*temp_freq
    
#     print()
#     print(f"freq_of_hypothesis_Spam  := {freq_of_hypothesis_Spam}")
#     print(f"freq_of_hypothesis_NOspam:= {freq_of_hypothesis_NOspam}")
    
    if freq_of_hypothesis_Spam > freq_of_hypothesis_NOspam:
        return print("email is SPAM")
    else:
        return print("email is NOT spam")

In [19]:
classify("Развивай бизнес на дому с услугой Безлимитный Интернет")

email is SPAM


In [20]:
    classify("Мы получили ваше сообщение о пропаже багажа и домашнего питомца в здании аэропорта. Конечно, нам жаль. Но что мы можем с этим сделать?")

email is NOT spam


In [21]:
classify("Перезвони по номеру +799999999 в течение 6 секунд и выиграй миллион рублей!")

email is NOT spam


In [22]:
# проверка Антона (2)
t = "Перезвони по номеру +799999999 в течение 6 секунд и выиграй миллион рублей!"
ws = set(re.sub(r"[!?]+$", "", elem) for elem in t.lower().strip().split(" "))

initiation_of_new_global_var(' '.join(ws))

sum([calculate_P_Bi_A(w,SPAM) for w in ws])

0.9999999999999998

In [23]:
sum([calculate_P_Bi_A(w,NOT_SPAM) for w in ws])

0.9999999999999998

In [24]:
# проверка Антона (2) на другом тексте
t = "Мы получили ваше сообщение о пропаже багажа и домашнего питомца в здании аэропорта. Конечно, нам жаль. Но что мы можем с этим сделать?"
ws = set(re.sub(r"[!?]+$", "", elem) for elem in t.lower().strip().split(" "))

initiation_of_new_global_var(' '.join(ws))

sum([calculate_P_Bi_A(w,SPAM) for w in ws])

0.9999999999999997

In [25]:
sum([calculate_P_Bi_A(w,NOT_SPAM) for w in ws])

0.9999999999999996

In [26]:
def classify_with_info(email):
    global alfa,N_spam,N_nospam,M,freq_of_spam,freq_of_NOspam
    temp_freq = calculate_P_B_A(email, 1)
    print(f"P_B_A in 1 := {temp_freq}")
    freq_of_hypothesis_Spam = freq_of_spam*temp_freq
    
    temp_freq = calculate_P_B_A(email, 0)
    print(f"P_B_A in 0 := {temp_freq}")
    freq_of_hypothesis_NOspam = freq_of_NOspam*temp_freq
    
    print()
    print(f"freq_of_hypothesis_Spam  := {freq_of_hypothesis_Spam}")
    print(f"freq_of_hypothesis_NOspam:= {freq_of_hypothesis_NOspam}")
    
    if freq_of_hypothesis_Spam > freq_of_hypothesis_NOspam:
        return print("email is SPAM")
    else:
        return print("email is NOT spam")

In [27]:
classify_with_info("Развивай бизнес на дому с услугой Безлимитный Интернет")

P_B_A in 1 := 5.960464477539063e-08
P_B_A in 0 := 4.6461146250837526e-08

freq_of_hypothesis_Spam  := 3.2514333724975585e-08
freq_of_hypothesis_NOspam:= 2.1116590971005657e-08
email is SPAM


In [28]:
classify_with_info("Мы получили ваше сообщение о пропаже багажа и домашнего питомца в здании аэропорта. Конечно, нам жаль. Но что мы можем с этим сделать?")

P_B_A in 1 := 7.19782746075171e-32
P_B_A in 0 := 9.578329374679561e-32

freq_of_hypothesis_Spam  := 3.9264148798400577e-32
freq_of_hypothesis_NOspam:= 4.353350700791861e-32
email is NOT spam


In [29]:
classify_with_info("Купите три по цене двух!")

P_B_A in 1 := 0.00038580246913580245
P_B_A in 0 := 0.00032000000000000013

freq_of_hypothesis_Spam  := 0.00021045524691358023
freq_of_hypothesis_NOspam:= 0.00014544000000000007
email is SPAM
